In [2]:
import numpy as np
import pandas as pd
import os
import random
from glob import glob
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from PIL import Image

In [3]:
""" 
This part of the code sets up an Image generator that will be used for the CNN
It doesn't actually compute any of the resizing or transformations yet, it just sets up the generator
The generator will be called when we train the model 
"""
from keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = (600, 400) # resolution of the images for the dataset.


def getNumImagesPerClass(dataset_dir='data/train', class_type='ALL', output=False):
    class_counts = {}

    for subdir in os.listdir(dataset_dir):
        if os.path.isdir(os.path.join(dataset_dir, subdir)):
            num_images = len(os.listdir(os.path.join(dataset_dir, subdir)))
            class_counts[subdir] = num_images

    if output:
        for class_name, count in class_counts.items():
            print(f"Class '{class_name}' has {count} images.")
    elif class_type in class_counts:
        return class_counts[class_type]
    else:
        return class_counts


getNumImagesPerClass(output=True)
getNumImagesPerClass(class_type="LAG")
value = getNumImagesPerClass(class_type="LAG")
print(f'value: {value}')

Class 'NoF' has 372 images.
Class 'OTHER' has 239 images.
Class 'DOL' has 183 images.
Class 'YFT' has 400 images.
Class 'LAG' has 163 images.
Class 'ALB' has 400 images.
Class 'BET' has 200 images.
Class 'SHARK' has 182 images.
value: 163


In [4]:
# TODO 
# Create a fish 'finder' 
# Expermienting with YOLO, Faster R-CNN, and SSD
    #Localization Model: This model focuses on finding and extracting the fish from the images.
    #Cropping: The identified fish regions are cropped to create high-resolution fish images.
    #Classification Model: This model classifies the cropped fish images to determine the fish species or other attributes.

In [5]:
""" 
The classses are not balanced! This code will augment the underepresented classes
classes = ['BET', 'DOL', 'LAG', 'OTHER', 'SHARK']
"""

# # Create an ImageDataGenerator random transformations to add more data
random_transform = ImageDataGenerator(
    rescale=1.0/255,
    horizontal_flip=True,
    vertical_flip=True, 
    rotation_range=20,
    brightness_range=[0.2, .5],
    channel_shift_range=0.2
)

def load_and_preprocess_image(img_path):
    img = Image.open(img_path)
    img = img.resize(IMAGE_SIZE)
    img_array = np.array(img)
    img_array = img_array / 255.0  # Normalize pixel values to the range [0, 1]
    return img_array

# # Create a saveImage function 
def save_image(image, file_path):
    # Create an Image object from the NumPy array
    image = Image.fromarray((image * 255).astype('uint8'))  # Assuming image values are in [0, 1] range
    image.save(file_path)


# # Function to randomly remove images from the class to prevent oversampling 
def remove_images_until_target(dataset_dir, target_num_images):
    file_list = os.listdir(dataset_dir)
    current_num_images = len(file_list)
    num_images_to_remove = max(current_num_images - target_num_images, 0)
    if num_images_to_remove == 0:
        print(f"No removal needed. The dataset already has {current_num_images} images.")
        return
    print(f"Removing {num_images_to_remove} images to reach the target of {target_num_images} images...")
    images_to_remove = random.sample(file_list, num_images_to_remove)
    for image_file in images_to_remove:
        image_path = os.path.join(dataset_dir, image_file)
        os.remove(image_path)


def createImages(minority_class, target_num_images):
    current_num_images = getNumImagesPerClass(class_type=minority_class)
    minority_class_dir = os.path.join('data/train', minority_class)
    for filename in os.listdir(minority_class_dir):
        if current_num_images >= target_num_images:
            break
        img_path = os.path.join(minority_class_dir, filename)
        img = load_and_preprocess_image(img_path)  # Load and preprocess your image
        img = random_transform.random_transform(img)  # Apply data augmentation using
        new_filename = f"augmented_{filename}"
        new_img_path = os.path.join(minority_class_dir, new_filename)
        save_image(img, new_img_path)  # Save the augmented image
        current_num_images += 1

TARGET_NUM_IMAGES = 200
classes = ['BET', 'DOL', 'LAG', 'OTHER', 'SHARK']
for image_class in classes:
    createImages(image_class, TARGET_NUM_IMAGES)
remove_images_until_target("data/train/ALB/", 500)
remove_images_until_target("data/train/YFT/", 500)
getNumImagesPerClass(output=True)

No removal needed. The dataset already has 400 images.
No removal needed. The dataset already has 400 images.
Class 'NoF' has 372 images.
Class 'OTHER' has 239 images.
Class 'DOL' has 185 images.
Class 'YFT' has 400 images.
Class 'LAG' has 175 images.
Class 'ALB' has 400 images.
Class 'BET' has 200 images.
Class 'SHARK' has 186 images.


In [6]:
"""
CREATE VALIDATION SET
The dataset should be unzipped into the folder named 'data' This folder should be in the same directory as this file.
Extract the train.zip file.
"""
print(os.listdir("data/")) # should list train.zip, sample_submission... etc 
# The Most important folder is going to be the train folder. 
# We will create our validation set by splitting the train folder into train and valid folders.
if not os.path.exists('data/valid'):
    os.mkdir('data/valid/')

# Now we will create validation folders for each type of fish 
fish_types = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT'] 
for fish in fish_types:
    # Creates a validation folder for each type of feature (fish)
    if not os.path.exists(f'data/valid/{fish}'):
        os.mkdir(f'data/valid/{fish}')

    # if valid folder already contains validation set, it breaks out of the loop
    if len(os.listdir(f'data/valid/{fish}')) > 0:
        print(f"Validation folder already contains files for {fish}")
        continue

    train_dir = f'data/train/{fish}/'
    valid_dir = f'data/valid/{fish}/'

    # List all files in the source directory
    file_paths = [os.path.join(train_dir, filename) for filename in os.listdir(train_dir)]

    # Use 20% of the files for validation, using the train_test_split function
    validation_ratio = 0.2
    train_files, valid_files = train_test_split(file_paths, test_size=validation_ratio, random_state=42)

    # Move the selected validation files to the validation directory
    for file_path in valid_files:
        filename = os.path.basename(file_path)
        destination = os.path.join(valid_dir, filename)
        os.rename(file_path, destination)

    print(f"{len(valid_files)} files from test/{fish} moved to validation set.")


# TODO
# Create a function to reset the validation, moving all validation images back into the test folder

['test_stg1', 'test_stg1.zip', 'train.zip', '__MACOSX', 'train', 'test_stg2.7z', 'valid', 'test_stg2', 'sample_submission_stg1.csv', 'the-nature-conservancy-fisheries-monitoring.zip']
Validation folder already contains files for ALB
Validation folder already contains files for BET
Validation folder already contains files for DOL
Validation folder already contains files for LAG
Validation folder already contains files for NoF
Validation folder already contains files for OTHER
Validation folder already contains files for SHARK
Validation folder already contains files for YFT


In [7]:
"""
DATAGENS 
"""
from sklearn.model_selection import train_test_split


# Create an ImageDataGenerator with data augmentation for training and normalization
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=40,
    brightness_range=[0.5, 1.5],
    channel_shift_range=0.2
)

# Create a separate ImageDataGenerator for validation with normalization only
val_datagen = ImageDataGenerator(rescale=1.0/255)

# Load and preprocess the training dataset
train_generator = train_datagen.flow_from_directory(
    'data/train/',
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical',
)

# Load and preprocess the validation dataset
validation_generator = val_datagen.flow_from_directory(
    'data/valid',
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical',
)


Found 2157 images belonging to 8 classes.
Found 500 images belonging to 8 classes.


In [8]:
"""
Test weighted classes to solve class imbalance
"""
from sklearn.utils.class_weight import compute_class_weight

classes = train_generator.classes
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(classes), y=classes)
class_weight_dict = dict(enumerate(class_weights))

In [9]:
"""
This part of the code sets up the CNN model
"""
from keras.layers import Input, Conv2D, MaxPooling2D, Dense, GlobalAveragePooling2D, AveragePooling2D
from keras.layers import Activation, BatchNormalization, Concatenate
from keras.models import Model
from keras.callbacks import EarlyStopping
import keras.regularizers as regularizers
from keras.applications import VGG16
from keras.applications import EfficientNetV2S
from keras.applications.efficientnet import preprocess_input
from keras.applications.vgg16 import preprocess_input
from keras.optimizers import Adam
from keras.regularizers import l2


# TODO 
# k-fold cross validation?
# A model to define 'fish' and then a model to identify that fish?

# Check if keras is using cpu or gpu
print(f"GPU: {tf.config.list_physical_devices('GPU')}")
print(tf.compat.v1.keras.backend)


# Define hyperparameters
input_shape = IMAGE_SIZE + (3,)
num_classes = 8


# VGG16 Model Setup with ImageNet weights
base_model = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=input_shape
)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.001))(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freezing layers in the base model
# for layer in base_model.layers:
#     layer.trainable = False


# Create and compile the model 
adam_optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])
print(model.summary())

# Stop the model if it's not improving 
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=10,         # Number of epochs with no loss improvement
    mode='min',          # 'min' for loss, 'max' for accuracy, 'auto' for automatic
    verbose=1,            # Set to 1 for a message when early stopping is triggered
    min_delta=0.005
)

# Run the model
model.fit(
    train_generator,
    steps_per_epoch=50, # num training samples (1600) / batchsize(32) = 50
    epochs=32,
    batch_size=32,
    validation_data=validation_generator,
    class_weight=class_weight_dict,
    validation_steps=10
    # callbacks=[early_stopping]
)

# Save the model
version = 'VGNewModel'
model.save(f'model{version}.keras')
print(f'model performance: {model.compiled_metrics.metrics}')

GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
<module 'keras.api._v1.keras.backend' from '/home/rashaka/.local/lib/python3.10/site-packages/keras/api/_v1/keras/backend/__init__.py'>


2023-11-04 15:26:53.979832: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-04 15:26:53.983088: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-04 15:26:53.983215: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 600, 400, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 600, 400, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 600, 400, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 300, 200, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 300, 200, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 300, 200, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 150, 100, 128)     0     

2023-11-04 15:26:58.071204: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-11-04 15:26:59.411930: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.11GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-11-04 15:26:59.714303: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.14GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-11-04 15:26:59.714339: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.34GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if m

50/50 [==============================] - 72s 1s/step - loss: 3.8185 - accuracy: 0.1290 - val_loss: 2.2659 - val_accuracy: 0.1000
Epoch 2/32
43/50 [========================>.....] - ETA: 8s - loss: 2.2543 - accuracy: 0.0987

KeyboardInterrupt: 

In [ ]:
"""
Display a confusion matrix
"""

from sklearn.metrics import confusion_matrix
import numpy as np

# Predict classes for the validation set
validation_generator.reset()
y_pred = model.predict(validation_generator, steps=len(validation_generator), verbose=1)
y_true = validation_generator.classes

# Get the class labels
class_labels = list(validation_generator.class_indices.keys())

# Create the confusion matrix
conf_matrix = confusion_matrix(y_true, np.argmax(y_pred, axis=1))

# Print the confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# To display the confusion matrix with class labels
print("Confusion Matrix with Class Labels:")
for i, row in enumerate(conf_matrix):
    print(f"{class_labels[i]}: {row}")

24/24 [==============================] - 6s 246ms/step
Confusion Matrix:
[[  1  49   9   3  35  46   3 198]
 [  0  10   0   1   4   4   0  21]
 [  0   2   0   0   4   2   2  14]
 [  0   2   0   0   0   0   0  12]
 [  1  13   1   0  14  11   2  51]
 [  0   6   0   0  13   7   3  31]
 [  0   5   2   0   3   7   0  19]
 [  1  23   2   0  21   8   7  85]]
Confusion Matrix with Class Labels:
ALB: [  1  49   9   3  35  46   3 198]
BET: [ 0 10  0  1  4  4  0 21]
DOL: [ 0  2  0  0  4  2  2 14]
LAG: [ 0  2  0  0  0  0  0 12]
NoF: [ 1 13  1  0 14 11  2 51]
OTHER: [ 0  6  0  0 13  7  3 31]
SHARK: [ 0  5  2  0  3  7  0 19]
YFT: [ 1 23  2  0 21  8  7 85]


In [ ]:
"""
Now run our trained model on the test set and create a submission csv file
"""
import keras
from tensorflow.keras.models import load_model
import sys


# Load the model 
model = load_model(f'model{version}.keras')

# Load the first test set, test_stg1
# Submission must be a combination of test_stg1 (1000) and test_stg2 (12153) = 13153
test_set1 = glob('data/test_stg1/*.jpg')
test_set2 = glob('data/test_stg2/*.jpg')

# Each image name must be called 'test_stg1/image_000001.jpg', for example
for i, image in enumerate(test_set1):
    test_set1[i] = image
for i, image in enumerate(test_set2):
    test_set2[i] = image
    # print(image) #'data/' + 'test_stg2/' + os.path.basename(image)
test_set = test_set1 + test_set2
# test_set=test_set1
print(test_set[0:10])

# Create a dataframe to hold the predictions
submission = pd.DataFrame(columns=['image', 'ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT'])

# For each image in the test set, run the models prediction and save it to the submission df
# It takes about a min to run on test_stg1 for me (Karl). 
# It takes about 15 min to run the combined test_stg1 and test_stg2
# for each iteration, it takes 20 to 40ms to run. 
for i, image in enumerate(test_set):
    # Preprocess the test images
    test_image = test_set[i]
    test_image = tf.keras.preprocessing.image.load_img(test_image, target_size=input_shape)
    test_image = tf.keras.preprocessing.image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    test_image = keras.applications.mobilenet.preprocess_input(test_image)

    # Run the prediction 
    prediction = model.predict(test_image)

    # Get the predicted class
    # The prediction array is a list of probabilities for each class
    # For each prediction, map the prediction to the name of the class
    # train_generator.class_indices is a dictionary of the classes of fish and their indices
    keys = list(train_generator.class_indices.keys())

    # Create a dictionary of the predictions
    # Looks like this: 
    #  {'ALB': 1.3623509e-07, 'BET': 9.1926294e-20,
    #   'DOL': 2.9418256e-16, 'LAG': 1.9241103e-13, 'NoF': 0.0019608391, 
    #   'OTHER': 1.6155835e-11, 'SHARK': 2.625621e-13, 'YFT': 0.99803907}
    prediction_dict = dict(zip(keys, prediction[0]))
    # print(prediction_dict)

    # Add the prediction to the dataframe
    if test_set[i].split("/")[1] == "test_stg2":
        submission.loc[i, 'image'] = os.path.join('test_stg2/', os.path.basename(test_set[i]))

    else: 
        submission.loc[i, 'image'] = os.path.basename(test_set[i])
    for key in keys:
        submission.loc[i, key] = prediction_dict[key]

    # Uncomment the break if you want to test 
    # break


# Print the head, does it look okay?
print(submission.columns)
print(submission.head())

# Save the dataframe to a csv file :)
submission.to_csv('submission.csv', index=False)



['data/test_stg1/img_02666.jpg', 'data/test_stg1/img_04949.jpg', 'data/test_stg1/img_01976.jpg', 'data/test_stg1/img_01966.jpg', 'data/test_stg1/img_03181.jpg', 'data/test_stg1/img_07149.jpg', 'data/test_stg1/img_03043.jpg', 'data/test_stg1/img_01983.jpg', 'data/test_stg1/img_06571.jpg', 'data/test_stg1/img_01618.jpg']
1/1 [==============================] - 0s 12ms/step
Index(['image', 'ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT'], dtype='object')
           image       ALB       BET       DOL       LAG       NoF     OTHER  \
0  img_02666.jpg  0.110118  0.025062  0.023566  0.000133  0.690986  0.003292   
1  img_04949.jpg  0.105091  0.724234  0.026154   0.00358  0.009706  0.022805   
2  img_01976.jpg  0.090439  0.284817  0.131751  0.000124  0.076791  0.004724   
3  img_01966.jpg  0.121886  0.431976  0.043753  0.142145  0.004799  0.110219   
4  img_03181.jpg  0.139758  0.331558  0.090775  0.037619  0.016426  0.216308   

      SHARK       YFT  
0  0.003003  0.143842  
1  0.

In [ ]:
import os

cur_dir = os.getcwd()
image_path = os.path.join(cur_dir, 'data', 'test_stg2', 'image_07462.jpg')

if os.path.exists(image_path):
    print(f"The file exists: {image_path}")
else:
    print(f"The file does not exist: {image_path}")


The file exists: /home/rashaka/Desktop/Programing/CodClassifier/data/test_stg2/image_07462.jpg
